In [ ]:
import random

# [Preparación] Algoritmos de Euclides

Antes de empezar necesitaremos un sistema para cálculo del máximo común divisor (mcd en castellano, gcd en inglés) de dos números, y el inverso de un número en un anillo cíclico. Ambas cosas se conocen desde hace tiempo: son dos "algoritmos de Euclides".

No os preocupéis de entender ningún algoritmo de esta sección, simplemente ejecutadlos porque los necesitaremos para el resto de ejercicios.

## Máximo común divisor

Algoritmo de Euclides para determinar el máximo común divisor (*gcd* por sus siglas en inglés) de dos enteros a y b

In [ ]:
def gcd(a, b):
    while b != 0:
        a, b = b, a % b
    return a

print('gcd(2, 3) = ', gcd(2, 3))
print('gcd(20, 30) = ', gcd(20, 30))
print('gcd(50720, 48184) = ', gcd(50720, 48184))


## Inverso multiplicativos de un número en un anillo cíclico $\mathbb{Z}_{\phi}$

Un inverso multiplicativo en un anillo cíclico en $\phi$ de un número $a$ es un número $a^{-1}$ tal que $a * a^{-1} \mod \phi = 1$

Recuerda que $\mod$ es la operación módulo, es decir, divide un número entre otro y quédate solo con el resto. 

Por ejemplo: $(3 * 5 \mod 7) = (15 \mod 7) = 1$ porque 15 entre 7 son 2 (que esto no nos importa) y **de resto** 1. Por tanto 5 es el inverso de 3 en $\mathbb{Z}_{7}$

In [ ]:
def multiplicative_inverse(e, phi):
    d = 0
    x1 = 0
    x2 = 1
    y1 = 1
    temp_phi = phi
    
    while e > 0:
        temp1 = temp_phi // e
        temp2 = temp_phi - temp1 * e
        temp_phi = e
        e = temp2
        
        x = x2 - temp1 * x1
        y = d - temp1 * y1
        
        x2 = x1
        x1 = x
        d = y1
        y1 = y

    if temp_phi == 1:
        return d + phi
    # no inverse: return None
    return None

print('3^{-1} mod 7 = ', multiplicative_inverse(3, 7))
print('3^{-1} mod 10 = ', multiplicative_inverse(3, 10))
print('2^{-1} mod 10 = ', multiplicative_inverse(2, 10))
print('25^{-1} mod 119 = ', multiplicative_inverse(25, 119))

## Test de si un número es primo

Algoritmo de testeo muy ineficiente de si un número es primo, pero que nos servirá en nuestros ejercicios porque usaramos números pequeñose.

In [ ]:
def is_prime(num):
    if num == 2:
        return True
    if num < 2 or num % 2 == 0:
        return False
    for n in range(3, int(num**0.5) + 2, 2):
        if num % n == 0:
            return False
    return True

for i in [2, 5, 19, 25, 222, 314, 317]:
    print(f'{i}: ', is_prime(i))

# RSA

Bien, ya tenemos todo lo necesario para poder hacer los ejercicios.

RSA es un algoritmo de cifrado asimétrico. Es decir, tiene dos claves: una para cifrar y otra diferente para descifrar. Puede usarse tanto para cifrar una información como para firmar digitalmente un documento.

RSA está compuestos por dos funciones sencillas:

- Una función que genera el par de claves necesario. Da como resultado dos claves, una se podrá hacer pública y la otra tiene que permanecer siempre privada.
- Una función que cifra, que es la misma función que para descifrar pero usando la otra clave.

Aquí vemos los dos algoritmos. Fíjate qué sencilla es la función para cifrar o descifrar.

In [ ]:
import random

def generate_keypair(p, q):
    if not (is_prime(p) and is_prime(q)):
        raise ValueError('Both numbers must be prime.')
    elif p == q:
        raise ValueError('p and q cannot be equal')
    #n = pq
    n = p * q

    #Phi is the totient of n
    phi = (p - 1) * (q - 1)

    # Choose an integer e such that e and phi(n) are coprime
    e = random.randrange(1, phi)

    # Use Euclid's Algorithm to verify that e and phi(n) are coprime
    g = gcd(e, phi)
    while g != 1:
        e = random.randrange(1, phi)
        g = gcd(e, phi)

    #Use Extended Euclid's Algorithm to generate the private key
    d = multiplicative_inverse(e, phi)
    
    #Return public and private keypair
    #Public key is (e, n) and private key is (d, n)
    return ((e, n), (d, n))

def encrypt(pk, number):
    # Unpack the key into it's components
    key, n = pk
    return (number ** key) % n

# The decrypt function is exactly the same than the encrypt function
decrypt = encrypt

 En los ejemplos usaremos como parámetros de configuración 17 y 23, dos números primos que simplemente sirven para configurar el algoritmo inicialmente. Cuanto más grandes sean, mayor será el tamaño en bits de la clave. Como estamos usando algoritmos poco eficientes para aprender, no uses números demasiado altos.

In [ ]:
pk, sk = generate_keypair(17, 23)
print(f'Clave pública pk=(e, n): {pk}')
print(f'Clave privada o secreta sk=(d, n): {sk}')

Fíjate: si generamos otro par de claves, aunque usemos los mismos primos, obtendremos unas claves diferentes. Eso es porque el parámetro $e$ se escoge al azar

In [ ]:
pk, sk = generate_keypair(17, 23)
print(f'Clave pública pk=(e, n): {pk}')
print(f'Clave privada o secreta sk=(d, n): {sk}')

Vamos a intentar cifrar un texto sencillo:

In [ ]:
print(encrypt(pk, 'hola'))

No podemos: RSA solo puede cifrar enteros. Una posibilidad es codificar el mensaje como un conjunto de enteros

In [ ]:
msg = [ord(c) for c in 'hola']
print(f'mensaje = {msg}')

c = [encrypt(pk, m) for m in msg]
print(f'cifrado = {c}')

¿Qué pasa si intentamos cifrar varias veces lo mismo?

In [ ]:
print([encrypt(pk, ord(c)) for c in 'aaaa'])

El texto cifrado es siempre igual. Pocas veces querremos eso. RSA debe usarse siguiendo recomendaciones como PKCS#1. Lo veremos un poco más abajo.

# (semi) Homorfismo

RSA es semihomomórfico con la multiplicación: se pueden hacer cálculos con los números cifrados, aunque no sepas lo que son ni qué resultado tienes. Al descifrar, el resultado es correcto. Más detalles: https://ciberseguridad.com/guias/prevencion-proteccion/criptografia/cifrado-homomorfico/

Por ejemplo, vamos a multiplicar los mensajes cifrados c1 y c2, que son los cifrados de 5 y 2 respectivamente

In [ ]:
m1 = 5
c1 = encrypt(pk, m1)
print(f'encrypt(pk, {m1}) = {c1}')
print(f'decrypt(sk, {c1}) = {decrypt(sk, c1)}')


In [ ]:
m2 = 2
c2 = encrypt(pk, m2)
print(f'encrypt(pk, {m2}) = {c2}')
print(f'decrypt(sk, {c2}) = {decrypt(sk, c2)}')

In [ ]:
cm = c1 * c2
print(f"c1 = {c1}; c2 = {c2}; cm = {cm}")

Un atacante no sabe cuánto vale c1 ni c2, ni sabe qué valor tiene cm, pero sabe que, sea lo que sea, ha multiplicado c1 y c2 y cuando se descifre el resultado va a ser correcto

In [ ]:
print(f'decrypt(sk, c1 * c2) = m1 * m2 = {m1} * {m2} = {decrypt(sk, cm)}')

Según la utilidad, el semihomorfismo puede ser útil o no:

- Sistemas PET (private enhanced technologies) necesitas calcular sin descifrar. Por ejemplo, voto electrónico
- Pero en general no querremos que un atacante pueda multiplicar una orden de pago por otro número y que el resultado sea válido: recomendaciones PKCS#1

# PyCryptoDome

La función de arriba es para jugar y solo sirve para ver cómo funciona RSA a alto nivel.

Vamos a usar PyCrytoDome, que incluye una librería RSA real.

Mide cuánto tiempo necesitamos para generar las claves ya más útiles de 2048 y 4096 bits. Prueba a generar claves mayores, de 16384 bits, por ejemplo, que es similar en seguridad al cifrado AES de 256 bits.

In [ ]:
!python3 -m pip install pycryptodome

# Clave de 2048 bits
from Crypto.PublicKey import RSA
key2048 = RSA.generate(2048)
key2048

In [ ]:
key4096 = RSA.generate(4096)
key4096

# Ejercicios

Hemos visto cómo crear claves con PyCryptoDrome, pero no cómo usarlo para cifrar o descifrar.

Recuerda de las transparencias que no es recomendable utilizar RSA "de forma pura", es decir, sin tener en cuenta muchas consideraciones sobre padding, conversiones, longitudes... que se recogen en [PKCS#1](https://en.wikipedia.org/wiki/PKCS_1). De hecho, PyCryptoDome no nos va a dejar utilizar el cifrado y descifrado directamente.

Observa que la línea siguiente da un error, avisando que uses el módulo  `Crypto.Cipher.PKCS1_OAEP`

In [ ]:
key2048.encrypt(b'hola', None)

**Aunque no se debe**, vamos a utilizar la función `_encrypt()`, que no está documentada pero la puedes encontrar en el código: https://github.com/Legrandin/pycryptodome/blob/master/lib/Crypto/PublicKey/RSA.py#L147

In [ ]:
c = key2048._encrypt(15)
d = key2048._decrypt(c)
print(f"Cifrado: {c}")
print(f"Descifrado: {d}")

Usando estas funciones `_encrypt()` y `_decrypt()` para cifrar cadenas:

1. Una posibilidad es cifrar cada caracter por separado y cifrarlos también por separado, como hemos hecho antes. ¿Cuándo ocupa el cifrado, en bytes?
1. Otra posibilidad es codificar la cadena como un enorme entero, es decir, cada caracter representa un byte de un número entero: `msg = int.from_bytes(b"hola mundo", "big")` ¿Cuánto ocupa el cifrado, en bytes?
1. ¿Puedes probar el método anterior para cifrar una cadena realmente larga, como `msg = int.from_bytes(b"hola mundo" * 1000, "big")` ? ¿Por qué crees que no funciona? ¿Cómo lo harías?

Vamos a hacer las cosas bien: cifra `"hola mundo"` y `"hola mundo" * 1000` usando PKCS1. Encontrarás en ejemplo en la documentación de pyCryptoDome: https://pycryptodome.readthedocs.io/en/latest/src/cipher/oaep.html

## Cifrado híbrido

En el tema de TLS veremos un cifrado híbrido: ciframos con RSA la clave AES que usamos para cifrar el texto.
1. Bob: Crea par de claves RSA
1. Alice: Crea clave simétrica AES. Cifra la clave AES con la clave pública de Bob. Envía mensaje
1. Alice: cifra "hola mundo" con clave AES. Envía mensaje
1. Bob: descifra clave AES con clave privada. Descifra mensaje de Alice

Entre los ejemplos de RSA precisamente verás algo así: https://pycryptodome.readthedocs.io/en/latest/src/examples.html#encrypt-data-with-rsa

- ¿Puedes hacer cifrado híbrido del mensaje "hola mundo"?
- ¿Se te ocurre por qué es necesario el cifrado híbrido?